Using the shot chart avaialable for each nba player via the nba api to create a machine learning routine that should be capable of predicting whether or not a basket will go in.

Variables include position, coverage, and time. 

Making it one big function that way as soon as you end the player, its done

In [12]:
from nba_py import player
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(4)  #replication purposes.
import matplotlib.pyplot as plt
from sklearn import preprocessing, cross_validation, neighbors
from sklearn.neural_network import MLPClassifier
from nba_py import shotchart
%matplotlib inline

In [17]:
players = player.PlayerList()

In [19]:
playersdf = players.info()

In [21]:
playersdf.head()

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG
0,203518,"Abrines, Alex",Alex Abrines,1,2016,2017,alex_abrines,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
1,203112,"Acy, Quincy",Quincy Acy,1,2012,2017,quincy_acy,1610612751,Brooklyn,Nets,BKN,nets,Y
2,203500,"Adams, Steven",Steven Adams,1,2013,2017,steven_adams,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
3,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2017,bam_adebayo,1610612748,Miami,Heat,MIA,heat,Y
4,201167,"Afflalo, Arron",Arron Afflalo,1,2007,2017,arron_afflalo,1610612753,Orlando,Magic,ORL,magic,Y


In [23]:
PIDs = playersdf['PERSON_ID'].valuues

In [26]:
len(PIDs)

548

In [27]:
for PID in PIDs:
    print(PID)

203518
203112
203500
1628389
201167
201582
202332
200746
1628386
1628443
2754
202329
1626147
203937
201583
1628387
203507
2546
1628384
1627853
2772
201589
1628503
201600
201571
202337
1628407
1627758
1627735
1628366
200826
203084
203115
201587
201573
203382
203145
203078
1627736
201563
201158
1628395
1627761
1627733
202722
201976
203920
1628444
202687
202357
204028
1628469
202339
203505
203992
202711
101106
1627762
1626164
202344
1628449
202340
1628396
201147
1627763
201166
1628415
202705
1626148
201628
1627759
203485
203900
1628425
1627852
1628418
203493
203504
202692
202710
202779
203998
101181
203484
203477
203991
201960
1713
203487
1627936
201956
1626161
1628463
201596
2199
201163
1627737
203546
203903
1628492
1628499
1628381
1628380
1627858
201954
2555
201144
1626192
1626188
1628429
204022
1627856
202326
203496
203459
1628470
2037
202348
203109
201967
203552
201939
203584
203076
1627738
202334
201942
203473
1626155
1627098
203521
2736
1627767
203476
203915
1628416
1628422
1628495


In [3]:
def get_PID(first_name,last_name):
    """Obtains the player ID number of the desired player.
    
    Parameters
    -----------
    
    first_name: str
        First name of player
    last_name: str
        Last name of player
    
    Returns
    -------
    
    ID : float
        The player's ID number. 
    """
    
    playa = player.get_player(first_name=first_name,last_name=last_name)
    
    return playa.values[0]
def get_shotchart(PID):
    """Returns a dataframe of the shotchart.
    """
    chart = shotchart.ShotChart(PID)
    chartdf = chart.shot_chart()
    
  #  importantstuff = chartdf[['LOC_X','LOC_Y','SHOT_DISTANCE','ACTION_TYPE','EVENT_TYPE']].copy()   #['LOC_X','LOC_Y','EVENT_TYPE']
    importantstuff = chartdf[['LOC_X','LOC_Y','ACTION_TYPE','EVENT_TYPE']].copy()
    importantstuff = pd.get_dummies(importantstuff)
    return importantstuff

In [10]:
def analyze_player(first,last,plot=True,cluster=True,Classifier=True):
    """Wrapper function which does everything in one, 
    no need to specify each table specific to that player right away!
    
    Parameters
    -----------
    first and last name of the player, comma separated, string format. 
    
    and a plotting option to see the shot chart, and anything else if I think of it. 
    and a cluster option to try and use ML to predict whether or not a shot goes in!
    
    Returns
    -------
    
    Shot chart, clustering success, anything else
    """
    
    
    PID = get_PID(first,last)
    print(first,last,'PID':,PID)
    playertable =  get_shotchart(PID)
    #print(playertable)
    if plot: 
        makes = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 0]
        misses = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 1]
        xmade = makes['LOC_X'].values/10
        ymade = makes['LOC_Y'].values/10
        xmissed = misses['LOC_X'].values/10
        ymissed = misses['LOC_Y'].values/10
        x = np.linspace(-60,60,100)
        #make a 3pt line for reference
        for i in range(len(x)):
            r = 22
            y = np.sqrt(r**2 - x**2)
        plt.plot(xmade,ymade,'mo',label = 'made')
        plt.plot(xmissed,ymissed,'yx',label = 'missed')
        plt.plot(x,y,'k-',label = '~3pt line')
        plt.xlim((-25, 25))  
        plt.ylim((0, 35))  

        plt.title("Shot Chart of " + first + " " + last)
        plt.xlabel('dist. from hoop (ft)')
        plt.ylabel('dist. from hoop (ft)')

        plt.legend()
        y = playertable[['EVENT_TYPE_Made Shot','EVENT_TYPE_Missed Shot']].values
        del playertable['EVENT_TYPE_Made Shot'],playertable['EVENT_TYPE_Missed Shot']
        X = playertable.values
    if cluster:
        X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
        clf = neighbors.KNeighborsClassifier()
        clf.fit(X_train,y_train)
        accuracy = clf.score(X_test,y_test)
        print("Accuracy of clustering:",100*accuracy,"%.")
    if Classifier:
        #try a NN, whatever
        #normalize rgith ere 
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        MinMaxScaler(copy=True, feature_range=(0, 1))

        scaler.fit(X)
        X = scaler.transform(X)
        X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
        model = MLPClassifier()
        model.shuffle = True
        model.fit(X_train,y_train)
        accuracy = model.score(X_test,y_test)

        print("Accuracy of MLP Classifier:",100*accuracy,"%.")


In [31]:
def analyze_players(plot=False,cluster=True,Classifier=True):
    """Wrapper function which does everything in one, 
    no need to specify each table specific to that player right away!
    

    Parameters
    -----------
    PIDSs. Array of all the active nba players, just to see which is best for fitting etc.     
    
    Returns
    -------
    
    Shot chart, clustering success, anything else
    """
    
    players = player.PlayerList() 
    
    playersdf = players.info()
    PIDs = playersdf['PERSON_ID'].values
    names = playersdf['DISPLAY_FIRST_LAST'].values
    for i in range(len(PIDs)):
        PID = PIDs[i]
        name = names[i]
        print(name)
        playertable =  get_shotchart(PID)
    #print(playertable)
        if plot: 
            makes = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 0]
            misses = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 1]
            xmade = makes['LOC_X'].values/10
            ymade = makes['LOC_Y'].values/10
            xmissed = misses['LOC_X'].values/10
            ymissed = misses['LOC_Y'].values/10
            x = np.linspace(-60,60,100)
        #make a 3pt line for reference
            for i in range(len(x)):
                r = 22
                y = np.sqrt(r**2 - x**2)
            plt.plot(xmade,ymade,'mo',label = 'made')
            plt.plot(xmissed,ymissed,'yx',label = 'missed')
            plt.plot(x,y,'k-',label = '~3pt line')
            plt.xlim((-25, 25))  
            plt.ylim((0, 35))  

            plt.title("Shot Chart of " + first + " " + last)
            plt.xlabel('dist. from hoop (ft)')
            plt.ylabel('dist. from hoop (ft)')

            plt.legend()
        y = playertable[['EVENT_TYPE_Made Shot','EVENT_TYPE_Missed Shot']].values
        del playertable['EVENT_TYPE_Made Shot'],playertable['EVENT_TYPE_Missed Shot']
        X = playertable.values
        if cluster:
            X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
            clf = neighbors.KNeighborsClassifier()
            clf.fit(X_train,y_train)
            accuracy = clf.score(X_test,y_test)
            print("Accuracy of clustering:",100*accuracy,"%.")
        if Classifier:
        #try a NN, whatever
        #normalize rgith ere 
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            MinMaxScaler(copy=True, feature_range=(0, 1))

            scaler.fit(X)
            X = scaler.transform(X)
            X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
            model = MLPClassifier()
            model.shuffle = True
            model.fit(X_train,y_train)
            accuracy = model.score(X_test,y_test)

            print("Accuracy of MLP Classifier:",100*accuracy,"%.")


In [32]:
analyze_players()

Alex Abrines
Accuracy of clustering: 50.847457627118644 %.


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Accuracy of MLP Classifier: 62.71186440677966 %.
Quincy Acy
Accuracy of clustering: 60.273972602739725 %.
Accuracy of MLP Classifier: 72.6027397260274 %.
Steven Adams
Accuracy of clustering: 66.43356643356644 %.
Accuracy of MLP Classifier: 70.62937062937063 %.
Bam Adebayo
Accuracy of clustering: 63.23529411764706 %.
Accuracy of MLP Classifier: 67.64705882352942 %.
Arron Afflalo
Accuracy of clustering: 36.36363636363637 %.
Accuracy of MLP Classifier: 69.6969696969697 %.
Alexis Ajinca


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


KeyError: "['EVENT_TYPE_Made Shot' 'EVENT_TYPE_Missed Shot'] not in index"

In [33]:
analyze_player('Alexis','Ajinca')

Alexis Ajinca PID 201582


KeyError: 'EVENT_TYPE_Missed Shot'

If I had to guess these lines are on the line, so approximately speaking 220 of these units corresponds to 22 feet. Ie divide by 10 to obtain feet. Doing this now!

In [ ]:
plt.plot(x,y)